# Wind Turbine Prognosis

## Importing the Libraries


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.signal import stft
from scipy.stats import kurtosis
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.utils import *

## Importing the Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


35 Samples of Medium Bearing Life Expectancy - Class: 0

15 Samples of Short Bearing Life Expectancy - Class: 1

In [ ]:
df0 = [] # List to Store the Data (Class 0)
df1 = [] # List to Store the Data (Class 1)

In [ ]:
# Looping over All Class 0 Samples
for i in range(35):
  df0.append(loadmat('/content/drive/MyDrive/data_DL_P/Class_0/'+ str(i+1)+'.mat')['vibration'].flatten())
df0 = np.asarray(df0) 

In [ ]:
# Looping over All Class 1 Samples
for i in range(15):
  df1.append(loadmat('/content/drive/MyDrive/data_DL_P/Class_1/'+ str(i+1)+'.mat')['vibration'].flatten())
df1 = np.asarray(df1) 

## Computing the Kurtosis Spectral Content

In [ ]:
f = [] # Storing the Sample Frequencies
fft = [] # Storing the stft Values
wnd = 127 # Length of Segment required for evaluating stft
fs = 97656 # Sampling Frequency (Hz)

In [ ]:
freq_cont = []

In [ ]:
for i in range(35):
    u,v,w = stft(df0[i],fs,nperseg = wnd,noverlap = int(0.8 * wnd),nfft = int(2.0 * wnd))
    f.append(u)
    fft.append(kurtosis(np.abs(w),fisher = False,axis = 1))
f = np.asarray(f)
fft = np.asarray(fft)

In [ ]:
f1 = [] 
fft1 = []

In [ ]:
for i in range(df1.shape[0]):
    u,v,w = stft(df0[i],fs,nperseg = wnd,noverlap = int(0.8 * wnd),nfft = int(2.0 * wnd))
    f1.append(u)
    fft1.append(kurtosis(np.abs(w),fisher = False,axis = 1))
f1 = np.asarray(f1)
fft1 = np.asarray(fft1)

Further Processing the fft values

In [ ]:
fftn = (fft - fft.min()) / (fft.max() - fft.min())
fftn = np.asarray(fftn)

In [ ]:
fftn1 = (fft1 - fft1.min()) / (fft1.max() - fft1.min())
fftn1 = np.asarray(fftn1)

In [ ]:
X = np.zeros((50,128)) # Input Variable

In [ ]:
X[0:35][:] = fftn

In [ ]:
X[35:50][:] = fftn1

## Creating input and output labels for Training the 1D CNN

In [ ]:
X = X.reshape(-1,f.shape[1],1) 
y = np.zeros(50,dtype = int) # True Output Labels
y[-15:] = 1
label = to_categorical(y)

## Defining the Model

In [ ]:
def create_model(dropout_rate):
  mdl = Sequential()
  mdl.add(Conv1D(100,6,activation = 'relu', input_shape = (128,1)))
  mdl.add(Conv1D(100,6,activation = 'relu'))
  mdl.add(MaxPooling1D(3))
  mdl.add(Conv1D(160,6,activation = 'relu'))
  mdl.add(Conv1D(160,6,activation = 'relu'))
  mdl.add(GlobalAveragePooling1D())
  mdl.add(Dropout(dropout_rate))
  mdl.add(Dense(2,activation = 'softmax'))
  mdl.compile(loss = 'categorical_crossentropy', optimizer='Adagrad', metrics=['accuracy'])
  return mdl

**Hyperparameter Tuning**

1. Epochs and Optimization Algorithm

In [ ]:
Xtrain,Xtest,ytrain,ytest = train_test_split(X,label,
                                             random_state = 1,
                                             test_size = 0.40)

In [ ]:
cw = {0: 5.0,1:75.0}

In [ ]:
!pip install scikeras[tensorflow]

In [ ]:
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier

In [ ]:
model = KerasClassifier(model=create_model,batch_size=30,verbose=0)
epochs = [100, 500, 1000]
optimizer = ['SGD','RMSprop', 'Adagrad','Adam']
param_grid = dict(optimizer=optimizer, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(Xtrain,ytrain,class_weight=cw)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
learn_rate = [0.01, 0.05, 0.1]
dropout_rate = [0.5, 0.6, 0.7]
model = KerasClassifier(model=create_model,batch_size=30,loss = 'categorical_crossentropy',epochs=500,verbose=0)
param_grid = dict(optimizer__learning_rate=learn_rate, model__dropout_rate=dropout_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(Xtrain,ytrain,class_weight=cw)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
mdl = create_model(dropout_rate=0.5)
mdl.compile(loss = 'categorical_crossentropy',
            optimizer = 'Adagrad',metrics = ['accuracy'])

In [ ]:
Xtest.shape

(20, 128, 1)

## Training

In [ ]:
batch,epoch = 30,500

op = mdl.fit(Xtrain,ytrain,batch_size = batch,
               epochs = epoch,validation_split = 0.2,verbose = 0,
               class_weight = cw)

## Testing

In [ ]:
mdl.evaluate(Xtest,ytest,verbose = 2)

## Other Metrics

In [ ]:
pred_op = []
actual_op = []

In [ ]:
pred_op = mdl.predict(Xtest)

1/1 [==============================] - 0s 105ms/step


In [ ]:
pred_op.shape

(20, 2)

In [ ]:
actual_op = np.argmax(ytest,axis=1)

In [ ]:
pred_op = np.argmax(pred_op,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(actual_op,pred_op)